In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.data_util import load_augmented_example, load_preprocess_examples
from src.evaluation import DAP_SAP_MAP_kde, hist_AP, plot_SAP_MAP

import tensorflow as tf

from sklearn.model_selection import train_test_split
import glob
import json
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import seaborn as sns
import pandas as pd

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
all_pigs = ["P_{0:02d}_PulHyp".format(i) for i in range(1, 11)]
print(all_pigs)

pig_test_s_dict = {
    "P_01_PulHyp": 10338,
    "P_02_PulHyp": 12859,
    "P_03_PulHyp": 8124,
    "P_04_PulHyp": 8914,
    "P_05_PulHyp": 10063,
    "P_06_PulHyp": 6934,
    "P_07_PulHyp": 5692,
    "P_08_PulHyp": 7189,
    "P_09_PulHyp": 8228,
    "P_10_PulHyp": 6552,
}

load_path = "/data/PulHypStudie_Check_npz_v4_SNR20/"

In [ ]:
X_train, y_train, clrs_pig_train = load_augmented_example(
    load_path, all_pigs, sample_skip=500, load_samples="upwards", shuffle=True
)

X_valid, y_valid, clrs_pig_valid = load_augmented_example(
    load_path, all_pigs, sample_skip=500, load_samples="downwards", shuffle=True
)

In [ ]:
print(
    X_train.shape,
    X_valid.shape,
    y_train.shape,
    y_valid.shape,
    clrs_pig_train.shape,
    clrs_pig_valid.shape,
)

In [ ]:
np.max(y_train, axis=0), np.max(y_valid, axis=0)

In [ ]:
dap_factor = 120
sap_factor = 180
map_factor = 150

y_train[:, 0] = y_train[:, 0] / dap_factor  # dap normalization
y_train[:, 1] = y_train[:, 1] / sap_factor  # sap normalization
y_train[:, 2] = y_train[:, 2] / map_factor  # map normalization

y_valid[:, 0] = y_valid[:, 0] / dap_factor  # dap normalization
y_valid[:, 1] = y_valid[:, 1] / sap_factor  # sap normalization
y_valid[:, 2] = y_valid[:, 2] / map_factor  # map normalization

In [ ]:
np.max(y_train, axis=0), np.max(y_valid, axis=0)

**Get test pig**

In [ ]:
for p_n, drop_pig in enumerate(all_pigs):
    print(f"Tuning for all pics and test on {drop_pig}")
    upw_num = pig_test_s_dict[drop_pig]
    print("Test smples:", upw_num)

In [ ]:
tmp = np.load(
    "/data/PulHypStudie_Check_npz_v4_SNR20/P_01_PulHyp/sample_014999.npz",
    allow_pickle=True,
)
tmp.files

In [ ]:
tmp["pig"]

In [ ]:
plt.hist(clrs_pig_train[:, 0])

In [ ]:
def find_pig_indices(pig_list: list, filt_for: str, mode="only"):
    pig_list = pig_list[:, 0]
    if mode == "only":
        idx = np.where(pig_list == filt_for)[0]
    if mode == "exclude":
        idx = np.where(pig_list != filt_for)[0]

    print(f"Found {len(idx)} entries.")
    return idx

In [ ]:
find_pig_indices(clrs_pig_train, filt_for="Pig_02")

In [ ]:
plt.figure(figsize=(13, 4))
plt.plot(y_train[:, 0])
plt.plot(y_train[:, 1])
plt.plot(y_train[:, 2])

In [ ]:
np.unique(clrs_pig_p1[:, 0])

In [ ]:
plt.figure(figsize=(13, 4))
plt.plot(y_p1[:, 0])
plt.plot(y_p1[:, 1])
plt.plot(y_p1[:, 2])

In [ ]:
clrs_pig_train

In [ ]:
X_test, y_test, clrs_pig_test = load_augmented_example(
    load_path, pigs, sample_skip=upw_num, load_samples="downwards", shuffle=True
)

In [ ]:
print(
    X_train.shape,
    y_train.shape,
    clrs_pig_train.shape,
)

In [ ]:
np.max(y_train, axis=0)

In [ ]:
dap_factor = 120
sap_factor = 180
map_factor = 150

y_train[:, 0] = y_train[:, 0] / dap_factor  # dap normalization
y_train[:, 1] = y_train[:, 1] / sap_factor  # sap normalization
y_train[:, 2] = y_train[:, 2] / map_factor  # map normalization

In [ ]:
np.max(y_train, axis=0)

**Drop Pig function**

In [ ]:
for p_n, drop_pig in enumerate(all_pigs):
    print(f"Tuning for all pics and test on {drop_pig}")
    # select train/test indices
    train_sel = find_pig_indices(
        clrs_pig_train, filt_for=f"Pig_0{p_n+1}", mode="exclude"
    )

# tbd:
# load only test pig non shuffled

X_test, y_test, clrs_pig_test = load_augmented_example(
    load_path, pigs, sample_skip=0, load_samples="upwards", shuffle=False
)